# Cuaderno 11: Funciones y tipos de datos especiales

La interfaz Python de Gurobi define funciones y tipos de datos (clases) especialmente diseñados para la formulación de modelos de programación matemática. Revisaremos en este cuaderno los tipos `tuplelist` y `tupledict`, y la función `multidict`.

Para utilizarlos es necesario importar el módulo de Gurobi:

In [2]:
from gurobipy import *

## La clase tuplelist

La clase `tuplelist` es un tipo especial de lista cuyos elementos son tuplas de longitud constante:

In [5]:
L = tuplelist([(1, 2), (1, 3), (1, 4), (2, 5), (3, 4), (4, 3)])
print(L)

<gurobi.tuplelist (6 tuples, 2 values each):
 ( 1 , 2 )
 ( 1 , 3 )
 ( 1 , 4 )
 ( 2 , 5 )
 ( 3 , 4 )
 ( 4 , 3 )
>


Un objeto de tipo `tuplelist` puede acceder a todas las funciones y métodos de una lista común:

In [6]:
L.append((3, 1))
print(L)
L.pop()
print(L)
L.insert(0, (1, 5))
print(L)

<gurobi.tuplelist (7 tuples, 2 values each):
 ( 1 , 2 )
 ( 1 , 3 )
 ( 1 , 4 )
 ( 2 , 5 )
 ( 3 , 4 )
 ( 4 , 3 )
 ( 3 , 1 )
>
<gurobi.tuplelist (6 tuples, 2 values each):
 ( 1 , 2 )
 ( 1 , 3 )
 ( 1 , 4 )
 ( 2 , 5 )
 ( 3 , 4 )
 ( 4 , 3 )
>
<gurobi.tuplelist (7 tuples, 2 values each):
 ( 1 , 5 )
 ( 1 , 2 )
 ( 1 , 3 )
 ( 1 , 4 )
 ( 2 , 5 )
 ( 3 , 4 )
 ( 4 , 3 )
>


Adicionalmente, un objeto `tuplelist`tiene un método `select` que retorna una sublista con las tuplas que cumplan determinados criterios:

In [7]:
print(L)
print(L.select(1, '*'))  # primera componente igual a 1
print(L.select([2,3], 4)) # primera componente igual a 2 ó 3, segunda componente igual a 4
print(L.select('*', [3, 4])) # segunda componente igual a 3 ó 4

<gurobi.tuplelist (7 tuples, 2 values each):
 ( 1 , 5 )
 ( 1 , 2 )
 ( 1 , 3 )
 ( 1 , 4 )
 ( 2 , 5 )
 ( 3 , 4 )
 ( 4 , 3 )
>
<gurobi.tuplelist (4 tuples, 2 values each):
 ( 1 , 5 )
 ( 1 , 2 )
 ( 1 , 3 )
 ( 1 , 4 )
>
<gurobi.tuplelist (1 tuples, 2 values each):
 ( 3 , 4 )
>
<gurobi.tuplelist (4 tuples, 2 values each):
 ( 1 , 3 )
 ( 4 , 3 )
 ( 1 , 4 )
 ( 3 , 4 )
>


Notar que la selección de tuplas del ejemplo anterior también podría hacerse con inclusiones de listas (*list comprehensions*). Sin embargo, el método `select` es computacionalmente mucho más eficiente.

In [8]:
# primera componente igual a 1
print([(i, j) for (i, j) in L if i==1])  
# primera componente igual a 2 ó 3, segunda componente igual a 4
print([(i, j) for (i, j) in L if i in [2,3] and j==4])  
# segunda componente igual a 3 ó 4
print([(i, j) for (i, j) in L if j in [3, 4]])  

[(1, 5), (1, 2), (1, 3), (1, 4)]
[(3, 4)]
[(1, 3), (1, 4), (3, 4), (4, 3)]


## La clase tupledict

La clase `tupledict` representa un tipo especial de diccionarios cuyas claves son tuplas de longitud constante. 

In [13]:
# ejemplo capacidades de los arcos de un grafo
u = tupledict({(1,2) : 3, 
               (1,3) : 4.5, 
               (1,4) : 5,
               (2,3) : 1.1,
               (4,3) : 0.33})
print(u)

{(1, 2): 3, (1, 3): 4.5, (1, 4): 5, (2, 3): 1.1, (4, 3): 0.33}


Pueden usarse todas las fuciones de un diccionario común. La lista de claves es un objeto del tipo `tuplelist`.

In [17]:
print(u[2,3])
print(u.keys())
u[1,4] = 6  # notar que los paréntesis no son necesarios para referirse a la clave
u[4,1] = 7  # es lo mismo que u[(4,1)] = 7
print(u)

1.1
<gurobi.tuplelist (6 tuples, 2 values each):
 ( 1 , 4 )
 ( 4 , 1 )
 ( 1 , 2 )
 ( 1 , 3 )
 ( 2 , 3 )
 ( 4 , 3 )
>
{(1, 2): 3, (1, 3): 4.5, (1, 4): 6, (2, 3): 1.1, (4, 3): 0.33, (4, 1): 7}


Como la lista de claves es un `tuplelist` puede emplearse la función `select` para iterar sobre ella: 

In [26]:
print('Nodos sucesores al 1 y sus capacidades:')
for i, j in u.keys().select(1, '*'):
    print('{}\t\t{}'.format(j, u[i,j]))


Nodos sucesores al 1 y sus capacidades:
4		6
2		3
3		4.5


Adicionalmente, la clase `tupledict` tienen dos métodos diseñados especialmente para facilitar la creación de expresiones lineales: `sum` y `prod`.

El método `sum` suma aquellos valores del diccionario cuyas claves satisfagan un criterio de selección:

In [27]:
print(u)
print(u.sum(1, '*'))  # sumar las capacidades de los arcos salientes de 1
print(u.sum(1, [2,3])) # sumar las capacidades de los arcos que salen de 1, y van a 2 ó 3

{(1, 2): 3, (1, 3): 4.5, (1, 4): 6, (2, 3): 1.1, (4, 3): 0.33, (4, 1): 7}
<gurobi.LinExpr: 13.5>
<gurobi.LinExpr: 7.5>


El método `prod` requiere de otro diccionario de coeficientes `c` que tenga las mismas claves que el diccionario actual `u`. Este método realiza las siguientes operaciones:
1. Para cada clave que satisface un criterio de selección, se multiplican los valores correspondientes de ambos diccionarios
2. Se suman los resultados de todos los productos


In [21]:
# ejemplo capacidades de los arcos de un grafo
c = tupledict({(1,2) : 2, 
               (1,3) : 1, 
               (1,4) : 2,
               (2,3) : 3,
               (4,3) : 4,
               (4,1) : 5})
# calcular c(1,2)*u(1,2) + c(1,3)*u(1,3) + c(1,4)*(1,4)
print("u = {}".format(u))
print("c = {}".format(c))
print(u.prod(c, 1, '*'))
# calcular c(1,2)*u(1,2) + c(1,3)*u(1,3) 
print(u.prod(c, 1, [2,3]))

u = {(1, 2): 3, (1, 3): 4.5, (1, 4): 6, (2, 3): 1.1, (4, 3): 0.33, (4, 1): 7}
c = {(1, 2): 2, (1, 3): 1, (1, 4): 2, (2, 3): 3, (4, 3): 4, (4, 1): 5}
<gurobi.LinExpr: 22.5>
<gurobi.LinExpr: 10.5>


Al igual que lo que ocurre con la función `select`, las funciones `sum` y `prod` podrían implementarse usando lazos, pero su ventaja radica en que son computacionalmente más eficientes.

## La función multidict

Suponer que se tiene un diccionario en el que los *valores* son siempre *tuplas de longitud constante* $k$:

In [22]:
D = {'a' : (1, 3, 5), 'b' : (2, 4, 6), 'c' : (1, -1, 0), 'd' : (4, 9, 8)} # cada valor es una tupla de longitud 3
print(D)

{'a': (1, 3, 5), 'b': (2, 4, 6), 'c': (1, -1, 0), 'd': (4, 9, 8)}


La función `multidict` se usa para separar este diccionario en una lista que contiene las claves de `D`, y $k$   diccionarios, cada uno formado por las claves y uno de los elementos de las tuplas:

In [24]:
(claves, primera, segunda, tercera) = multidict(D)
print(claves)
print(primera)
print(segunda)
print(tercera)

['a', 'b', 'c', 'd']
{'a': 1, 'b': 2, 'c': 1, 'd': 4}
{'a': 3, 'b': 4, 'c': -1, 'd': 9}
{'a': 5, 'b': 6, 'c': 0, 'd': 8}


Las claves del diccionario original pueden ser a su vez tuplas. En este caso, la lista de claves es del tipo `tuplelist` que se describe en la próxima sesión. Esto es útil en la formulación de muchos modelos de optimización. 

In [25]:
# Diccionario con datos de entrada:
# arcos : capacidades, costos
datos = {(1,2) : (3, 5), 
         (1,3) : (4, 4), 
         (1,4) : (5, 3),
         (2,3) : (1, 1),
         (4,3) : (0, 7)}
arcos, capacidades, costos = multidict(datos)
print(arcos)
print(capacidades)
print(costos)

<gurobi.tuplelist (5 tuples, 2 values each):
 ( 1 , 2 )
 ( 1 , 3 )
 ( 1 , 4 )
 ( 2 , 3 )
 ( 4 , 3 )
>
{(1, 2): 3, (1, 3): 4, (1, 4): 5, (2, 3): 1, (4, 3): 0}
{(1, 2): 5, (1, 3): 4, (1, 4): 3, (2, 3): 1, (4, 3): 7}


## Recordatorio: Expresiones generadoras

Las expresiones generadoras (o simplemente generadores) son parte de la sintaxis regular del lenguaje Python. Permiten construir listas (u otros iterables) "sobre la marcha", generalmente para utilizarlas en funciones. 

Los generadores emplean la misma sintaxis que las inclusiones (*list comprehensions*).

Ejemplo: Dados

In [26]:
u = tupledict({(1,2) : 3, 
               (1,3) : 4.5, 
               (1,4) : 5,
               (2,3) : 1.1,
               (2,4) : 0.33})
d = tupledict({1 : 1, 2: 2, 3 : 2, 4 : 7})
print(d)
print(u)

{1: 1, 2: 2, 3: 2, 4: 7}
{(1, 2): 3, (1, 3): 4.5, (1, 4): 5, (2, 3): 1.1, (2, 4): 0.33}


Suponer que queremos calcular el valor de $\sum_{(2,j)} d_j u_{2,j} = d_3 u_{2,3} + d_4 u_{2,4}$. 

Una posibilidad es usar inclusiones para construir una lista con los términos del sumatorio, y luego llamar a la función `quicksum` para sumar los elementos de la lista:

In [33]:
L = [d[j]*u[i,j] for i,j in u.keys().select(2, '*')]
print(L)
print(quicksum(L))

[2.2, 2.31]
<gurobi.LinExpr: 4.51>


Otra posibilidad es usar directamente un generador como argumento de la función `quicksum`:

In [27]:
print(quicksum(d[j]*u[i,j] for i,j in u.keys().select(2, '*')))

<gurobi.LinExpr: 4.51>
